# DeepSequence: SKU-Level Forecasting Demo

This notebook demonstrates how to use DeepSequence, a Prophet-inspired deep learning architecture for SKU-level forecasting.

## Overview

DeepSequence combines:
- **Seasonal Components**: Captures weekly, monthly, and yearly seasonality
- **Regression Components**: Models trends and exogenous variables
- **Deep Learning**: Leverages neural networks for complex pattern recognition

**Author**: Mritunjay Kumar  
**Year**: 2025

## 1. Setup and Imports

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Add src to path
sys.path.insert(0, os.path.join(os.getcwd(), '..', 'src'))

# Import DeepSequence components
from deepsequence import (SeasonalComponent, RegressorComponent, 
                       DeepSequenceModel, create_time_features, 
                       prepare_data, train_val_test_split)
from deepsequence.config import DATA_DIR, MODEL_DIR
from deepsequence.activations import CUSTOM_ACTIVATIONS

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

print("✓ All imports successful!")

## 2. Load and Prepare Data

In [ ]:
# Load data
data_path = os.path.join(DATA_DIR, 'stock_week_cluster.csv')
data = pd.read_csv(data_path)

# Drop unnecessary columns
if 'Unnamed: 0' in data.columns:
    data.drop(['Unnamed: 0'], axis=1, inplace=True)

# Convert date
data['ds'] = pd.to_datetime(data['ds'])

print(f"Data shape: {data.shape}")
print(f"Date range: {data['ds'].min()} to {data['ds'].max()}")
data.head()

## 3. Feature Engineering

In [ ]:
# Create time features
data = create_time_features(data, date_col='ds')

# Filter data (remove 2009 as it's anomalous)
data = data[data['year'] >= 2010].copy()

# Prepare data with standardization and lags
data = prepare_data(
    data,
    target_col='Quantity',
    id_col='StockCode',
    standardize=True,
    create_lags=True,
    lag_periods=[1, 4, 52]
)

print("✓ Feature engineering complete!")
print(f"Features: {data.columns.tolist()}")

## 4. Train/Validation/Test Split

In [ ]:
# Split data
train, val, test = train_val_test_split(
    data,
    date_col='ds',
    val_weeks=8,
    test_weeks=0  # No test set for now, using val as test
)

print(f"Train: {train.shape} | Val: {val.shape}")
print(f"Train date range: {train['ds'].min()} to {train['ds'].max()}")
print(f"Val date range: {val['ds'].min()} to {val['ds'].max()}")

## 5. Encode Categorical Variables

In [ ]:
from deepfuture.utils import encode_categorical

# Encode StockCode
train, encoder = encode_categorical(train, 'StockCode')
val, _ = encode_categorical(val, 'StockCode', encoder=encoder)

# Rename for model
train['id_cat'] = train['StockCode_encoded']
val['id_cat'] = val['StockCode_encoded']

print("✓ Categorical encoding complete!")

## 6. Prepare Model Inputs

In [ ]:
# Define feature groups
categorical_features = ['week_no', 'wom', 'cluster']
target = ['tQuantity']
id_var = 'id_cat'

# Prepare time series data
train_ts = train[['id_cat', 'ds', 'tQuantity']].copy()
val_ts = val[['id_cat', 'ds', 'tQuantity']].copy()

# Prepare exogenous variables
exog_cols = ['price', 'holiday', 'lag1', 'lag4', 'lag52'] + categorical_features
train_exog = train[exog_cols].copy()
val_exog = val[exog_cols].copy()

# Context variables (continuous)
context_vars = [col for col in exog_cols if col not in categorical_features]

print(f"Categorical features: {categorical_features}")
print(f"Context variables: {context_vars}")

## 7. Build DeepSequence Model

In [ ]:
from deepsequence.utils import encode_categorical

# Initialize Seasonal Component
seasonal = SeasonalComponent(
    data=train_ts,
    target=target,
    id_var=id_var,
    horizon=8,
    weekly=True,
    monthly=True,
    yearly=True,
    unit='w'
)

# Create seasonal features
seasonal.seasonal_feature()

# Build seasonal model
seasonal.seasonal_model(
    hidden=1,
    hidden_unit=4,
    hidden_act=CUSTOM_ACTIVATIONS['mish'],
    output_act=CUSTOM_ACTIVATIONS['swish'],
    reg=0.011,
    embed_size=50,
    drop_out=0.1
)

print("✓ Seasonal component built!")
seasonal.s_model.summary()

In [ ]:
# Initialize Regressor Component
regressor = RegressorComponent(
    ts=train_ts,
    exog=train_exog,
    target=target,
    id_var=id_var,
    categorical_var=categorical_features,
    context_variable=context_vars,
    constraint=None,
    embed_size=50,
    lat_unit=4,
    lattice_size=4,
    hidden_unit=4,
    hidden_act=CUSTOM_ACTIVATIONS['mish'],
    output_act=CUSTOM_ACTIVATIONS['listh'],
    hidden_layer=1,
    drop_out=0.1,
    L1=0.032,
    rnge=0.8
)

# Build regressor model (using seasonal ID input for shared embedding)
regressor.reg_model(id_input=seasonal.s_model.input[-1])

print("✓ Regressor component built!")
regressor.combined_reg_model.summary()

In [ ]:
# Combine into DeepSequence
model = DeepSequenceModel(mode='additive')
model.build(seasonal, regressor)

print("✓ DeepSequence assembled!")
model.summary()

## 8. Prepare Training Inputs

In [ ]:
def prepare_model_inputs(seasonal_comp, regressor_comp, ts_data, exog_data, val=False):
    """
    Prepare inputs for DeepSequence model.
    """
    seasonal_inputs = []
    regressor_inputs = []
    
    # Get seasonal features
    if val:
        sr_df = seasonal_comp.seasonal_feature(ts_data)
    else:
        sr_df = seasonal_comp.sr_df
    
    # Seasonal inputs
    for col in sr_df.columns:
        if col not in ['ds']:
            seasonal_inputs.append(sr_df[col].values)
    
    # Regressor inputs
    for input_name in regressor_comp.input_names:
        regressor_inputs.append(exog_data[input_name].values)
    
    return [seasonal_inputs, regressor_inputs]

# Prepare training and validation inputs
train_inputs = prepare_model_inputs(seasonal, regressor, train_ts, train_exog, val=False)
val_inputs = prepare_model_inputs(seasonal, regressor, val_ts, val_exog, val=True)

# Get targets
train_y = train['tQuantity'].values
val_y = val['tQuantity'].values

print(f"✓ Inputs prepared!")
print(f"Train samples: {len(train_y)}")
print(f"Val samples: {len(val_y)}")

## 9. Train the Model

In [ ]:
# Compile model
model.compile(loss='mape', learning_rate=0.001)

# Train model
checkpoint_path = os.path.join(MODEL_DIR, 'deepsequence_best.h5')

history = model.fit(
    train_input=train_inputs,
    train_target=train_y,
    val_input=val_inputs,
    val_target=val_y,
    epochs=500,
    batch_size=512,
    checkpoint_path=checkpoint_path,
    patience=10,
    verbose=1
)

print("\n✓ Training complete!")

## 10. Visualize Training History

In [ ]:
# Plot training history
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Model Loss (MAPE)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history.history['val_loss'], label='Val Loss', color='orange')
plt.title('Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('MAPE')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

print(f"Best validation MAPE: {min(history.history['val_loss']):.2f}%")

## 11. Make Predictions

In [ ]:
# Predict on validation set
val_pred = model.predict(val_inputs)

# Add predictions to validation data
val['forecast'] = val_pred

print("✓ Predictions generated!")
val[['StockCode', 'ds', 'tQuantity', 'forecast']].head(10)

## 12. Evaluate Performance

In [ ]:
from deepsequence.utils import calculate_mape

# Calculate MAPE for non-zero values
val_nonzero = val[val['tQuantity'] != -1].copy()
mape = calculate_mape(val_nonzero['tQuantity'], val_nonzero['forecast'])

print(f"Overall Validation MAPE: {mape:.2f}%")

# Per-SKU performance
sku_mape = val_nonzero.groupby('StockCode').apply(
    lambda x: calculate_mape(x['tQuantity'], x['forecast'])
).reset_index(name='mape')

print(f"\nTop 10 SKUs by MAPE:")
print(sku_mape.nsmallest(10, 'mape'))

## 13. Visualize Sample Forecasts

In [ ]:
# Select a few SKUs for visualization
sample_skus = val['StockCode'].unique()[:6]

fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.flatten()

for i, sku in enumerate(sample_skus):
    sku_data = data[data['StockCode'] == sku].sort_values('ds')
    sku_val = val[val['StockCode'] == sku].sort_values('ds')
    
    axes[i].plot(sku_data['ds'], sku_data['tQuantity'], 
                label='Historical', alpha=0.7)
    axes[i].plot(sku_val['ds'], sku_val['forecast'], 
                label='Forecast', color='red', linewidth=2)
    axes[i].set_title(f'SKU: {sku}')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 14. Save Model and Predictions

In [ ]:
from deepsequence.config import FORECAST_DIR

# Save forecasts
forecast_path = os.path.join(FORECAST_DIR, 'deepsequence_forecast.csv')
val[['StockCode', 'ds', 'Quantity', 'forecast']].to_csv(forecast_path, index=False)

print(f"✓ Forecasts saved to: {forecast_path}")
print(f"✓ Model saved to: {checkpoint_path}")

## Summary

This notebook demonstrated:
1. ✅ Data loading and preprocessing
2. ✅ Feature engineering (time features, lags, standardization)
3. ✅ Building DeepSequence architecture
4. ✅ Training with early stopping and model checkpointing
5. ✅ Evaluation and visualization
6. ✅ Generating forecasts

### Next Steps
- Compare with LightGBM models
- Hyperparameter tuning with Optuna
- Ensemble multiple models
- Deploy for production use